#Enhancing AI Responses with Few-Shot Prompting and Google GenAI
## Made by: Wilfredo Aaron Sosa Ramos (AI Lab Manager at RealityAI Labs)

In [4]:
!pip install langchain langchain_core langchain_community langchain_google_genai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 24.3 MB/s eta 0:00:

##Setting Up the Environment


In [2]:
import os
from google.colab import userdata

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

##Defining the Example Selector with Google Generative AI Embeddings


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma

# Define sample examples
examples = [
    {"question": "What is the capital of France?", "answer": "Paris"},
    {"question": "Who wrote '1984'?", "answer": "George Orwell"},
    {"question": "What is the boiling point of water?", "answer": "100°C or 212°F"},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci"},
]

# Initialize embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create an example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma
)

##Creating a Few-Shot Prompt Template


In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate

# Define the example format
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Q: {question}\nA: {answer}"
)

# Create a few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Answer the following questions:",
    suffix="Q: {input}\nA:",
    input_variables=["input"]
)

##Generating a Response with Google Generative AI


In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Input question
input_question = "Who is the author of 'Pride and Prejudice'?"

# Format the prompt with the input
formatted_prompt = few_shot_prompt.format(input=input_question)

# Generate the response
response = llm.invoke(formatted_prompt)

print(response)


content='Jane Austen\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-dc10dab5-614d-41e6-a0f2-963338576b29-0' usage_metadata={'input_tokens': 94, 'output_tokens': 3, 'total_tokens': 97, 'input_token_details': {'cache_read': 0}}


In [11]:
response.content.strip()

'Jane Austen'

In [18]:
# Input question
input_question = "Who are the founders of Google?"

# Format the prompt with the input
formatted_prompt = few_shot_prompt.format(input=input_question)

# Generate the response
response2 = llm.invoke(formatted_prompt)

print(response2)

content='Larry Page and Sergey Brin\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-c88004c5-2990-42d3-9dce-bb6ecc300355-0' usage_metadata={'input_tokens': 91, 'output_tokens': 6, 'total_tokens': 97, 'input_token_details': {'cache_read': 0}}


In [19]:
response2.content.strip()

'Larry Page and Sergey Brin'